In [1]:
!pip install openpyxl

In [2]:
from IPython.display import HTML, Math, Latex, Markdown, display

import itertools
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_rows', 80)

# Індивідуальне завдання

1. Виконати процедуру першого етапу модифікованого морфологічного аналізу, знайти ймовірності альтернатив і зробити відповідні висновки;
2. Виконати процедуру другого етапу модифікованого морфологічного аналізу, знайти очікувані результативності альтернатив параметру рішення і зробити відповідні висновки.

В результаті експертного опитування були отримані експертні думки щодо ймовірностей альтернатив параметрів (табл. 6) і значень матриці взаємозв'язків (табл. 7), а також значення матриці зв'язків параметрів 1−3 і параметра 4 (табл. 10). 

In [4]:
def make(header, rows):
    df = pd.DataFrame([[0 for x in range(len(header.index))] for x in range(len(rows.index))])
    df.index = rows.index
    df.columns = header.index
    return df

def combinations(*df):
    combinations = []
    for i1 in df[0].index:
        for i2 in df[1].index:
            for i3 in df[2].index:
                combinations.append([i1,i2,i3])
    return combinations

table = "<table><tbody><tr>{}</tr></tbody></table>".format('<td style="vertical-align:top">{}</td>'*3)

### Табл. 6. Попередні оцінки ймовірностей, отримані від експертів

In [5]:
times = [f"1.{x}" for x in range(1, 3)]  
df_times = pd.DataFrame(np.array([.7, .1]), columns=["Час"], index=times)
times_html = df_times.to_html()


places = [f"2.{x}" for x in range(1, 6)] 
df_places = pd.DataFrame(np.array([.2, .3, .6, .6, .8]), columns=["Місце"], index=places)
places_html = df_places.to_html()

reasons = [f"3.{x}" for x in range(1, 9)] 
df_reasons = pd.DataFrame(np.array([.6, .8, .7, .1, .1, .05, .3, .05]), columns=["Причини"], index=reasons)
reasons_html = df_reasons.to_html()

display(HTML(table.format(times_html, places_html, reasons_html)))

,Час
1.1,0.7
1.2,0.1
,Місце
2.1,0.2
2.2,0.3
2.3,0.6
2.4,0.6
2.5,0.8
,Причини
3.1,0.60


### Табл. 7. Матриця взаємозв'язків параметрів таблиці

In [6]:
columns = list(df_times.index) + list(df_places.index)
rows = list(df_places.index) + list(df_reasons.index)
data = np.zeros((len(rows), len(columns)))
df_cor = pd.DataFrame(data, columns=columns, index=rows)
 
#  row/ column / value
times_places =   [ (1, 1, .1), (2, 1,  .1), (3, 1,  .1), (4, 1, .1), (5, 1,  .5), ]
times_reasons =  [ (1, 1, .5), (6, 1, -.5), (1, 2, -.3), (2, 2, .3), (3, 2,  .3), (5, 2,  .3), (6, 2, .5), (7, 2, .3),]
places_reasons = [ (1, 5, .3), (2, 5, -.3), (3, 5, -.3), (4, 5, .3), (5, 5, -.3), (6, 5, -.1), ] 


for i in times_places: 
    df_cor.loc[f'2.{i[0]}', f'1.{i[1]}'] = i[2]

for i in times_reasons: 
    df_cor.loc[f'3.{i[0]}', f'1.{i[1]}'] = i[2]
    
for i in places_reasons: 
    df_cor.loc[f'3.{i[0]}', f'2.{i[1]}'] = i[2]
    
display(HTML(df_cor.to_html().replace("0.0", "")))

,1.1,1.2,2.1,2.2,2.3,2.4,2.5
2.1,0.1,,,,,,
2.2,0.1,,,,,,
2.3,0.1,,,,,,
2.4,0.1,,,,,,
2.5,0.5,,,,,,
3.1,0.5,-0.3,,,,,0.3
3.2,,0.3,,,,,-0.3
3.3,,0.3,,,,,-0.3
3.4,,,,,,,0.3
3.5,,0.3,,,,,-0.3


# Етап 1 

## Нормуємо дані морфологічної оцінки експертів за 3 критеріями.

In [7]:
df_times['Час'] /= df_times['Час'].sum()
df_places['Місце'] /= df_places['Місце'].sum()
df_reasons['Причини'] /= df_reasons['Причини'].sum()


times_html = df_times.to_html()
places_html = df_places.to_html()
reasons_html = df_reasons.to_html()

display(HTML(table.format(times_html, places_html, reasons_html)))

,Час
1.1,0.875
1.2,0.125
,Місце
2.1,0.08
2.2,0.12
2.3,0.24
2.4,0.24
2.5,0.32
,Причини
3.1,0.222222


In [8]:
# display(Markdown("### Морфологічна конфігурація"))


def C(row):
    
    a = df_cor.loc[row['Місце'], row['Час']]+1
    b = df_cor.loc[row['Причини'], row['Час']]+1
    c = df_cor.loc[row['Причини'], row['Місце']]+1

    return a*b*c

def Norm(row):
    time = df_times.loc[row['Час'], 'Час']
    place = df_places.loc[row['Місце'], 'Місце']
    reason = df_reasons.loc[row['Причини'], 'Причини']
    
    return time * place * reason

df_morf=pd.DataFrame(combinations(df_times, df_places, df_reasons), columns=["Час", "Місце", "Причини"])
df_morf['С\''] = df_morf.apply(C, axis=1)
df_morf['С'] = df_morf.apply(Norm, axis=1)
df_morf['P\'s'] = df_morf['С\''] * df_morf['С']
df_morf['Ps'] = df_morf['P\'s'] / df_morf['P\'s'].sum()

df_morf.to_excel("table_1_new.xlsx")

# df_morf

In [9]:
def fill(df, morf):
    for idx in df.index:
        df.loc[idx, df.columns[0]] = morf[(morf[df.columns[0]] == idx)]['Ps'].sum()

df_times_r = df_times.copy()
fill(df_times_r, df_morf)
times_html = df_times_r.to_html()

df_places_r =df_places.copy() 
fill(df_places_r, df_morf)
places_html = df_places_r.to_html()
 
df_reasons_r = df_reasons.copy()
fill(df_reasons_r, df_morf)
reasons_html = df_reasons_r.to_html()

display(Markdown("### Табл. 9. Ймовірності альтернатив (отримана за результатами Таблиці 1)"))
display(HTML(table.format(times_html, places_html, reasons_html)))

### Табл. 9. Ймовірності альтернатив (отримана за результатами Таблиці 1)

,Час
1.1,0.893892
1.2,0.106108
,Місце
2.1,0.074556
2.2,0.111834
2.3,0.223667
2.4,0.223667
2.5,0.366276
,Причини
3.1,0.326040


### Проміжні висновки

Згідно результатів прорахунків Таблиці 1:
* Найбільш вагомою альтернативою дорожніх заторів по параметру «Часу» з оцінкою експертів 0,7 є 1шою з ймовірністю 0.893. 
* За параметром «Місце» найбільшою є 5 з попередньою оцінкою експертів в 0,8 з ймовірністю 0.3662. 3 та 4 альтернативи є однаковими і становлять 0.223667.  Найгіршою альтернативою є 1.
* В параметрі «Причина» найвагомішою є 1 з ймовірністю 0.326, найбільш близькими до неї є 2 і 3 альтернатива.  Найменшими є альтернатива 6

 ## Етап 2

In [13]:
display(Markdown("## Табл. 10. Матриця зв'язків параметрів 1−3 і параметра 4"))
indexes = list(itertools.chain(list(df_times.index), list(df_places.index), list(df_reasons.index)))

df = pd.DataFrame(np.zeros((len(indexes),5)), 
                index=indexes,
                columns=[ f"4.{x}" for x in range(1,6)])

values = [ ('1.1', '4.5',  .5), 
           ('1.2', '4.3', -.3), 
           ('2.1','4.1',   .5), ('2.1', '4.5', .5),
           ('2.2', '4.1', -.7), 
           ('2.3', '4.1', -.7), 
           ('2.4', '4.1', -.5), ('2.4', '4.2', .3),
           ('2.5', '4.2',  .5), ('2.5', '4.3', .5), ('2.5', '4.4', .5), ('2.5', '4.5', .5),
           ('3.1', '4.1',  .7), ('3.1', '4.2', .5), ('3.1', '4.3', .5), ('3.1', '4.4',-.5),
           ('3.2', '4.1',  .3), ('3.2', '4.5',-.3),
           ('3.3', '4.1',  .3), ('3.3', '4.3',-.3), ('3.3', '4.4',-.3), ('3.3', '4.5',-.3),
           ('3.5', '4.1', -.5),
           ('3.7', '4.1', -.3),
         ]

for i in values: 
    df.loc[f'{i[0]}', f'{i[1]}'] = i[2]

display(HTML(df.to_html().replace("0.0", "")))

## Табл. 10. Матриця зв'язків параметрів 1−3 і параметра 4

,4.1,4.2,4.3,4.4,4.5
1.1,,,,,0.5
1.2,,,-0.3,,
2.1,0.5,,,,0.5
2.2,-0.7,,,,
2.3,-0.7,,,,
2.4,-0.5,0.3,,,
2.5,,0.5,0.5,0.5,0.5
3.1,0.7,0.5,0.5,-0.5,
3.2,0.3,,,,-0.3
3.3,0.3,,-0.3,-0.3,-0.3


In [11]:
def calc(key):
    def _calc_(row):
        a = df.loc[row['Час'],   key]
        b = df.loc[row['Місце'], key]   
        c = df.loc[row['Причини'], key] 
        return (a+1)*(b+1)*(c+1)
    return _calc_

def norm(key):
    def _norm_(row):
        return row[f"$R_{key}^{{4'}}$"] / row["$R_1^{4'}$":"$R_5^{4'}$"].sum()
    return _norm_

df_2nd = df_morf[['Час','Місце', 'Причини', 'Ps' ]].copy()

df_2nd["$R_1^{4'}$"] = df_2nd.apply(calc('4.1'), axis=1)
df_2nd["$R_2^{4'}$"] = df_2nd.apply(calc('4.2'), axis=1)
df_2nd["$R_3^{4'}$"] = df_2nd.apply(calc('4.3'), axis=1)
df_2nd["$R_4^{4'}$"] = df_2nd.apply(calc('4.4'), axis=1)
df_2nd["$R_5^{4'}$"] = df_2nd.apply(calc('4.5'), axis=1)
df_2nd["$R_1^{4}$"] = df_2nd.apply(norm('1'), axis=1)
df_2nd["$R_2^{4}$"] = df_2nd.apply(norm('2'), axis=1)
df_2nd["$R_3^{4}$"] = df_2nd.apply(norm('3'), axis=1)
df_2nd["$R_4^{4}$"] = df_2nd.apply(norm('4'), axis=1)
df_2nd["$R_5^{4}$"] = df_2nd.apply(norm('5'), axis=1)

df_2nd["$R_1^{4}Ps$"] = df_2nd["$R_1^{4}$"] * df_2nd['Ps']
df_2nd["$R_2^{4}Ps$"] = df_2nd["$R_2^{4}$"] * df_2nd['Ps']
df_2nd["$R_3^{4}Ps$"] = df_2nd["$R_3^{4}$"] * df_2nd['Ps']
df_2nd["$R_4^{4}Ps$"] = df_2nd["$R_4^{4}$"] * df_2nd['Ps']
df_2nd["$R_5^{4}Ps$"] = df_2nd["$R_5^{4}$"] * df_2nd['Ps']

df_2nd.to_excel("table_2_new.xlsx")

### В результаті (Таблиця 2) обрахунків маємо наступну картину 

In [12]:
pd.DataFrame({
    '4.1': ['0.1415', '5'],
    '4.2': ['0.2435', '2'],
    '4.3': ['0.2054', '3'],
    '4.4': ['0.1586', '4'],
    '4.5': ['0.2511', '1'],
}, index=["Значення", "Ранг"]).T

,Значення,Ранг
4.1,0.1415,5
4.2,0.2435,2
4.3,0.2054,3
4.4,0.1586,4
4.5,0.2511,1


З таблиці 2 спостерігаємо, що найсуттєвішими очікуваними результативності альтернатив є для альтернатив 5 та 2, що досягають значень 0,2511 та 0,2435 відповідно. Наближеним, але трохи меншим є результативність для 3 альтернативи – 0,2054. Гіршими залишаються 4 та 1 з показниками 0,1415 та 0,1586 відповідно. 